# Recolección datos de Twitter

Rate limites: https://developer.twitter.com/en/docs/basics/rate-limits.html

## Conexión con la API

In [ ]:
import configparser  
import sys
from tweepy import API # conda install -c conda-forge tweepy 
from tweepy import OAuthHandler

def get_twitter_auth():
    """Configura la autenticación con Twitter

    Retorna: un objeto tweepy.OAuthHandler
    """
    config = configparser.ConfigParser()
    config.read("d://notebooks//webmining//twitter.ini")
    try:
        consumer_key = config.get("TwitterKeys", "ConsumerKey")
        consumer_secret = config.get("TwitterKeys", "ConsumerSecret")
        access_token = config.get("TwitterKeys", "AccessToken")
        access_secret = config.get("TwitterKeys", "AccessTokenSecret")
    except:
        print("exception on %s!" % option)
        #sys.exit(1)  
        
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    return auth

def get_twitter_client():
    """Configura un cliente de la API de Twitter.

    Retorna: objeto tweepy.API
    """
    auth = get_twitter_auth()
    client = API(auth, wait_on_rate_limit=True)
    return client

auth = get_twitter_auth()
client = API(auth)
print("Se puede otorgar acceso a esta aplicación con el siguiente link: ")
print(auth.get_authorization_url(),"\n\n")


El método **get_twitter_auth** es el encargado de la autenticación de la aplicación.

**Os** posee un diccionario environ con las variables de entorno, que se acceden por clave. Si alguna de las variables falta, surge una **KeyError**

El método **get_twitter_client** se usa para crear una instancia de la API de Tweepy


## Leer tweets de los diferentes timeline

### Home timeline

Lo primero que podemos probar es obtener los primeros 10 tweets de nuestro propio **home timeline**.

El home timeline es lo que vemos cuando ingresamos a twitter, y contiene una secuencia de tweets de los usuarios que seguimos (los más recientes primero)

In [ ]:
from tweepy import Cursor

client = get_twitter_client()
for status in Cursor(client.home_timeline).items(10):
    # Procesamos un Status a la vez
    print(status.text,"\n")

**tweepy.Status** es el modelo usado por Tweepy para encapsular los tweets. En este caso accedemos al texto, pero tiene otros datos como veremos más adelante.

**tweepy.Cursor** es un objeto iterable que facilita la iteración y paginación de resultados



In [ ]:
import json
from tweepy import Cursor

with open('d://notebooks//webmining//home_timeline.jsonl', 'w') as f:
    for page in Cursor(client.home_timeline, count=200).pages(4):
        for status in page:
            f.write(json.dumps(status._json)+"\n")

En este ejemplo iteramos 4 paginas de 200 tweets. La razón es por una limitación de Twitter: solo podemos recuperar los 800 tweets más recientes de nuestro home timeline.

El formato jsonl es JSON LINES, representa un JSON valido por cada línea, útil para procesar grandes volúmenes de datos.

### User timeline

In [ ]:
user = "pottermore"
client = get_twitter_client()
fname = "d://notebooks//webmining//user_timeline_{}.jsonl".format(user)
page_no = 1
with open(fname, 'w') as f:
    for page in Cursor(client.user_timeline, screen_name=user, count=200).pages(16):
        print("Descargando página {} con {} tweets".format(page_no, len(page)))    
        page_no+=1
        for status in page:
            f.write(json.dumps(status._json)+"\n")
            


In [ ]:
import json
tweet = client.user_timeline(client.user_timeline, screen_name=user, count = 2)[0]
print(json.dumps(tweet._json, indent=4, sort_keys=True))

Las variables enteras tienen también una variante en formato string para los lenguajes que no soportan enteros de 64 bits.

**entities** es un diccionario que contiene diferentes entidades reconocidas en el tweet, como por ejemplo hashtags, fotos, urls y menciones a usuario

Se puede observar que toda la información del usuario está contenida dentro del mismo tweet (redundancia).

### Public timeline

In [ ]:
#Función auxiliar para imprimir con formato
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
client = get_twitter_client()
fname = "d://notebooks//webmining//public_timeline_query.jsonl"
with open(fname, 'w') as f:
    for tweet in Cursor(client.search,q="#Rusia2018",count=100,lang="en",since="2018-01-01").items(10): 
        f.write(json.dumps(tweet._json)+"\n")
        printmd("Tweet de <b>{}</b>: {}".format(tweet.user.screen_name, tweet.text))
        print("-------------------")


## La API de streaming

In [ ]:
import sys
import string
import time
from tweepy import Stream
from tweepy.streaming import StreamListener

La siguiente clase extiende de StreamListener y redefine dos métodos: **on_data()** y **on_error()** que son handlers que se ejecutan cuando llegan nuevos datos y cuando surge un error, respectivamente. Ambos métodos retornan un booleano indicando si se debe continuar escuchando el stream (true) o si ocurrió un error fatal y se debe interrumpir la escucha (false)

Puede chequearse la lista de errores en la documentación de la API de Twitter: 

https://developer.twitter.com/en/docs/basics/response-codes


In [ ]:
class CustomListener(StreamListener):
    """Custom StreamListener for streaming Twitter data."""
    def __init__(self, fname, start_time, limit):
        safe_fname = format_filename(fname)
        self.outfile = "d://notebooks//webmining//stream_{}_{}.jsonl".format(limit, safe_fname)        
        self.limit = limit
        self.start_time = start_time
        self.num_tweets = 0
    
    def on_data(self, data):
        if (self.num_tweets < self.limit):
            self.num_tweets += 1
            try:
                with open(self.outfile, 'a') as f:
                    f.write(data)
                    return True
            except BaseException as e:
                sys.stderr.write("Error on_data: {}\n".format(e))
                time.sleep(5)
                return True
        else:
            print ("{} tweets descargados en {} segs".format(self.num_tweets, time.time()-start_time))
            return False
        
    def on_error(self, status):
        if status == 420:
            sys.stderr.write("Rate limit exceeded\n")
            return False
        else:
            sys.stderr.write("Error {}\n".format(status))
            return True
        

Utilizamos dos funciones auxiliares para eliminar caracteres inválidos de los nombres de archivos y reemplazarlos por _

In [ ]:
def format_filename(fname):
    """Convert fname into a safe string for a file name.

    Return: string
    """
    return ''.join(convert_valid(one_char) for one_char in fname)


def convert_valid(one_char):
    """Convert a character into '_' if "invalid".

    Return: string
    """
    valid_chars = "-_.%s%s" % (string.ascii_letters, string.digits)
    if one_char in valid_chars:
        return one_char
    else:
        return '_'

In [ ]:
query = ["\#Rusia2018", "WorldCup"] # list of arguments
query_fname = ' '.join(query) # string
auth = get_twitter_auth()
start_time = time.time()
twitter_stream = Stream(auth, CustomListener(query_fname, start_time, limit=100))
twitter_stream.filter(track=query, async=True, languages=["en"])

In [ ]:
twitter_stream.disconnect()

## Análisis de tweets

Podemos obtener, por ejemplo, los 20 hashtags más frecuentes. 

La clase **Counter**, es una subclase de **Dict** que se encarga de contar la cantidad de ocurrencias de cada clave.

Usamos una función auxiliar **get_hashtags** que recibe un tweet completo y obtiene los hashtags. En lugar de acceder directamente al atributo del tweet con **tweet['entities']** usamos el método get al que podemos indicarle el valor por defecto en caso de que el atributo no se encuentre

In [ ]:
import sys
from collections import Counter
import json

def get_hashtags(tweet):
    entities = tweet.get('entities', {})
    hashtags = entities.get('hashtags', []) 
    # cada hashtags tiene la  siguiente estructura {'text': 'Herbology', 'indices': [127, 137]}
    # solo nos interesa el text
    return [tag['text'].lower() for tag in hashtags]

fname = "user_timeline_pottermore.jsonl"
fname = "stream_10000___Rusia2018_WorldCup_saved.jsonl"
with open(fname, 'r') as f:
    hashtags = Counter() 
    for line in f:
        tweet = json.loads(line) #Leemos la linea del jsonl (un json en si misma)
        hashtags_in_tweet = get_hashtags(tweet) # obtenemos los hashtags del tweet
        hashtags.update(hashtags_in_tweet) # actualizamos el Counter
    for tag, count in hashtags.most_common(20):
        print("{}: {}".format(tag, count))

Podemos obtener una estadística más descriptiva con el siguiente script

In [ ]:
from collections import defaultdict

with open(fname, 'r') as f:
    hashtag_count = defaultdict(int)
    
    for line in f:
        tweet = json.loads(line)
        hashtags_in_tweet = get_hashtags(tweet)
        n_of_hashtags = len(hashtags_in_tweet)
        hashtag_count[n_of_hashtags] += 1
        
    tweets_with_hashtags = sum([count for n_of_tags, count in hashtag_count.items() if n_of_tags > 0])
    tweets_no_hashtags = hashtag_count[0]
    tweets_total = tweets_no_hashtags + tweets_with_hashtags
    print("Cantidad de tweets")
    tweets_with_hashtags_percent = "%.2f" % (tweets_with_hashtags / tweets_total * 100)
    tweets_no_hashtags_percent = "%.2f" % (tweets_no_hashtags / tweets_total * 100)
    print("{} tweets sin hashtags ({}%)".format(tweets_no_hashtags, tweets_no_hashtags_percent))
    print("{} tweets con al menos un  hashtag ({}%)".format(tweets_with_hashtags, tweets_with_hashtags_percent))
    
    for tag_count, tweet_count in hashtag_count.items():
        if tag_count > 0:
            percent_total = "%.2f" % (tweet_count / tweets_total * 100)
            percent_elite = "%.2f" % (tweet_count / tweets_with_hashtags * 100)
            print("{} tweets con {} hashtags ({}% del total de tweets, {}% del total con hashtags)".format(tweet_count, tag_count, percent_total, percent_elite))

De forma similar podemos observar las menciones a diferentes usuarios

In [ ]:
def get_mentions(tweet):
    entities = tweet.get('entities', {})
    hashtags = entities.get('user_mentions', [])
    return [tag['screen_name'] for tag in hashtags]

In [ ]:
with open(fname, 'r') as f:
    users = Counter()
    for line in f:
        tweet = json.loads(line)
        mentions_in_tweet = get_mentions(tweet)
        users.update(mentions_in_tweet)
    for user, count in users.most_common(20):
        print("{}: {}".format(user, count))

## Análisis de texto

Vamos a utilizar NLTK para analizar el texto de los tweets

In [ ]:
import sys
import string
import json
from collections import Counter
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

Por cada tweet recibido (solo el texto), usamos el TweetTokenizer para separarlo en palabras y filtramos stopwords recibidas por parámetro

In [ ]:
def process(text, tokenizer=TweetTokenizer(), stopwords=[]):
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    return [tok for tok in tokens if tok not in stopwords and not tok.isdigit()]

Creamos el tokenizador de Tweets (recordar clase anterior) y usamos las stopwords comunes de ingles de nltk, más los signos de puntuación, más un conjunto de stopwords del dominio

In [ ]:
tweet_tokenizer = TweetTokenizer()
punct = list(string.punctuation)
stopword_list = stopwords.words('english') + punct + ['rt', 'via', '...', '…']

Por cada tweet levantado del archivo jsonl, lo procesamos y actualizamos la cantidad de veces que aparece cada palabra. Luego imprimimos las 30 palabras más frecuentes

In [ ]:
fname = "user_timeline_pottermore.jsonl"
tf = Counter()
with open(fname, 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tokens = process(text=tweet.get('text', ''),
                         tokenizer=tweet_tokenizer,
                         stopwords=stopword_list)
        tf.update(tokens)
    for tag, count in tf.most_common(30):
        print("{}: {}".format(tag, count))

Como podemos ver, el resultado incluye una mezcla de palabras, menciones a usuarios, ulrs y simbolos no incluidos como puntuación. Estos simbolos se pueden eliminar simplemente extendiendo la lista de stopwords.

Podemos extender los scripts anteriores generando un gráfico de frecuencias con matplotlib

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

tweet_tokenizer = TweetTokenizer()
punct = list(string.punctuation)
stopword_list = stopwords.words('english') + punct + ['rt', 'via', '...', '…']

fname = "user_timeline_pottermore.jsonl"
tf = Counter()
with open(fname, 'r') as f:
        for line in f:
            tweet = json.loads(line)
            tokens = process(text=tweet.get('text', ''),
                             tokenizer=tweet_tokenizer,
                             stopwords=stopword_list)
            tf.update(tokens)
        y = [count for tag, count in tf.most_common(1000)]
        x = range(1, len(y)+1)
        plt.bar(x, y)
        plt.title("Term Frequencies")
        plt.ylabel("Frequency")
        plt.show()

Se puede observar que hay pocos términos que aparecen muchas veces (a la izquierda)

# Análisis de series temporales

En términos generales, una serie temporal es una secuencia de datos que consisten en sucesivas observaciones a lo largo de un período de tiempo. Como Twitter provee para cada tweet el atributo **created_at** podemos ordenar los tweets en el tiempo para estudiar, por ejemplo, cómo los usuario reaccionan a eventos en tiempo real (por ejemplo eventos deportivos, conciertos, elecciones políticas, catátstrofes, programas televisivos, etc.).

Otra posible aplicación es estudiar qué opinan los usuarios acerca de un producto o una marca.

Como nos interesa estudiar la masa de usuarios y no un usuario individual, conviene utilizar la api de Streaming

In [ ]:
import sys
import json
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import pandas as pd
import numpy as np
import pickle

In [ ]:
fname = "stream___Rusia2018_WorldCup_saved.jsonl"
fname = "stream_10000___Rusia2018_WorldCup_saved.jsonl"
with open(fname, 'r') as f:
    all_dates = []
    #i=1
    for line in f:
        #print("reading line ",i)
        #i+=1
        tweet = json.loads(line)
        all_dates.append(tweet.get('created_at'))
    
    ones = np.ones(len(all_dates)) # un arreglo de todos unos
    idx = pd.DatetimeIndex(all_dates) #serie de datos, indexada por las fechas observadas en los tweets    
    my_series = pd.Series(ones, index=idx) # la serie en si (por el momento, todos unos)

    # Resampleo/agrupamiento en ventanas de 1 minuto
    per_minute = my_series.resample('1Min').sum().replace(np.nan, 0)
    print(my_series.head())
    print(per_minute.head())

    fig, ax = plt.subplots()
    ax.grid(True)
    ax.set_title("Tweet Frequencies")

    hours = mdates.MinuteLocator(interval=20)
    date_formatter = mdates.DateFormatter('%H:%M')

    #datemin = datetime(2018, 5, 22, 20, 32)
    #datemax = datetime(2018, 5, 22, 20, 40)

    ax.xaxis.set_major_locator(hours)
    ax.xaxis.set_major_formatter(date_formatter)
    #ax.set_xlim(datemin, datemax)
    max_freq = per_minute.max()
    ax.set_ylim(0, max_freq)
    ax.plot(per_minute.index, per_minute)
    
    plt.rcParams["figure.figsize"] = [20,9]
    plt.show()

## Users, Followers y Comunidades

Una de las grandes diferencias entre Twitter y otros medios sociales (por ejemplo Facebook o LinkedIn) es la forma en la que los usuarios se conectan entre sí. Las relaciones entre los usuarios de Twitter son unidireccionales. Esto quiere decir que un usuario puede conectarse con otros sin necesidad de que éstos últimos acepten la conexión.
En Twitter los usuarios con los que un usuario A se conecta se denominan **friends** mientras que los que se conectan con el usuario A se denominan **followers** desde el punto de vista de A. Cuando la conexión es bidireccional hablamos de **mutual friends**



## Estructura de un usuario

Una de las posibilidades básicas de la API de Twitter es ver el perfil de un usuario dado. Esta actividad está limitada actualmente a 900 requests cada 15 minutos.

El método **get_user** de la API, recibe un nombre de usuario y retorna un objeto del tipo **tweepy.models.User**


In [ ]:
import json
client = get_twitter_client()
profile = client.get_user(screen_name="PotterMore")
print(json.dumps(profile._json, indent=4))

## La "red" de un usuario

Se pueden ver las listas de followers y de amigos con **API.followers()** y **API.friends()** pero hay una limitación muy grande de 15 request cada 15 minutos donde cada request sólo retorna hasta 20 perfiles de usuario. Esto nos da una limitación de 300 perfiles cada 15 minutos, que hace imposible usar esta técnica para descargar grandes volúmenes de datos.

Un *truco* es utilizar **API.followers_ids** y **API.friends_ids** que retorna grupos de 5000 IDs por request (un total de 75000 cada 15 minutos). Con este método solo tendremos los identificadores de usuario, pero luego podemos usar estos ID para obtener la información de los usuarios con el método **lookup** que toma listas de hasta 100 IDs como entrada y retorna los perfiles completos como salida. El método **lookup** tiene una limitación de 300 pedidos cada 15 minutos, lo que nos da un total de 30000 perfiles cada 15 minutos.

In [ ]:
import os
import sys
import json
import time
import math
from tweepy import Cursor

MAX_FRIENDS = 1500

screen_name = "mg_armentano"
client = get_twitter_client()
dirname = "users/{}".format(screen_name)

max_pages = math.ceil(MAX_FRIENDS / 5000)
try:
    os.makedirs(dirname, mode=0o755, exist_ok=True)
except OSError:
    print("El directorio {} ya existe".format(dirname))
except Exception as e:
    print("Error al crear el directorio {}".format(dirname))
    print(e)
    sys.exit(1)
    
def paginate(items, n):
    """Genera bloques de tamaño n a partir de los items"""
    for i in range(0, len(items), n):
        yield items[i:i+n]

### Obtener los seguidores del usuario

In [ ]:
fname = "users/{}/followers.jsonl".format(screen_name)
with open(fname, 'w') as f:
    for followers in Cursor(client.followers_ids, screen_name=screen_name).pages(max_pages):
        for chunk in paginate(followers, 100):
            print("Descargando 100 followers de un total de ",len(followers))
            users = client.lookup_users(user_ids=chunk)
            for user in users:
                f.write(json.dumps(user._json)+"\n")            
            if len(followers) == 5000:
                print("Hay más resultados, pero esperamos 60 segundos para evitar el rate limit")
                time.sleep(60)            
        print("Página terminada!")
    print("Todo descargado!")

### Muy similar para los amigos

In [ ]:
fname = "users/{}/friends.jsonl".format(screen_name)
with open(fname, 'w') as f:
    for friends in Cursor(client.friends_ids, screen_name=screen_name).pages(max_pages):
        for chunk in paginate(friends, 100):
            print("Descargando 100 friends de un total de ",len(friends))
            users = client.lookup_users(user_ids=chunk)
            for user in users:
                f.write(json.dumps(user._json)+"\n")
            print("Bloque terminado!")
            if len(friends) == 5000:
                print("Hay más resultados, pero esperamos 60 segundos para evitar el rate limit")
                time.sleep(60)            
        print("Página terminada!")
    print("Todo descargado!")

### Obtenemos el perfil de cada usuario

In [ ]:
user_name="mg_armentano"
followers_file = "users/{}/followers.jsonl".format(user_name)
with open(followers_file) as f1:
    for line in f1:
        profile = json.loads(line)
        screen_name = profile['screen_name']
        print("Descargando {}".format(screen_name))
        
        fname = "users/{}.json".format(screen_name)
        with open(fname, 'w') as f:
            profile = client.get_user(screen_name=screen_name)
            f.write(json.dumps(profile._json, indent=4))

## Análisis de la red descargada

Una vez que tenemos descargada la inforamción de la red, podemos realizar algunas estadísticas

In [ ]:
import sys
import json

screen_name = "mg_armentano"
followers_file = 'users/{}/followers.jsonl'.format(screen_name)
friends_file = 'users/{}/friends.jsonl'.format(screen_name)
with open(followers_file) as f1, open(friends_file) as f2:
    followers = []
    friends = []
    for line in f1:
        profile = json.loads(line)
        followers.append(profile['screen_name'])
    for line in f2:
        profile = json.loads(line)
        friends.append(profile['screen_name'])
    mutual_friends = [user for user in friends if user in followers]
    followers_not_following = [user for user in followers if user not in friends]
    friends_not_following = [user for user in friends if user not in followers]
    print("{} tiene {} seguidores".format(screen_name, len(followers)))
    print("{} tiene {} amigos".format(screen_name, len(friends)))
    print("{} tiene {} amigos mutuos".format(screen_name, len(mutual_friends)))
    print("{} amigos no siguen a {}".format(len(friends_not_following), screen_name))
    print("{} seguidores no son seguidos por {}".format(len(followers_not_following), screen_name))

## Comunidades

En la vida real, una comunidad es un grupo de personas con algo en común. Esta definición tan general incluye, por ejemplo, comunidades de gente que trabaja en determinada area geográfica, gente con intereses políticos similares, gente de la misma religión, gente que comparte un interés común, etc.

En las redes sociales online también surgen comunidades de este tipo cuando gente con intereses comunes comienzan a interactuar.

Las comunidades pueden ser **explícitas**, cuando cada miembro sabe exactamente si pertenece o no a una comunidad y generalmente conoce quiénes son los demás miembros de la comunidad, o **implícitas** cuando las comunidades surgen simplemente por las interacciones entre los miembros.

Una de las técnicas utilizadas para descubrir comunidades implícitas es el **clustering** de usuarios. Podemos por ejemplo, agrupar los seguidores de un usuario utilizando la descripción dada en sus perfiles.

In [ ]:
import sys
import json
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

connections_file="users/mg_armentano/followers.jsonl"
with open(connections_file) as f:
    users = []
    # Carga de datos
    for line in f:
        profile = json.loads(line)
        users.append(profile['description'])

    # Creación de los vectores de términos
    vectorizer = TfidfVectorizer(max_df=0.8, 
                                 min_df=2, 
                                 max_features=200, 
                                 stop_words='english', 
                                 ngram_range=(1,3), 
                                 use_idf=True)
    X = vectorizer.fit_transform(users)
    print("Dimensiones de los datos: {}".format(X.shape))

    # Aprendizaje del modelo usando K-Means
    km = KMeans(n_clusters=5)
    km.fit(X)
    clusters = defaultdict(list)
    for i, label in enumerate(km.labels_):
        clusters[label].append(users[i])

    # imprimimos 10 descripciones de usuarios de cada cluster
    for label, descriptions in clusters.items():
        print('---------- Cluster {}'.format(label+1))
        for desc in descriptions[:10]:
            print(desc)

## Conversaciones

En Twitter, los usuarios pueden interactuar respondiendo a un tweet particular. Cuando dos o más usuarios utilizan este mecanismo, se genera una "conversación". Las conversaciones generan un grafo dirigido acíclico (DAG) por lo que podemos aplicar algoritmos de grafos sobre ellas. 

Por ejemplo, el **grado** de un nodo es la cantidad de nodos en el grafo que son hijos del mismo. Desde el punto de vista de una conversación, estamos hablando de la cantidad de respuestas recibidas. Otro concepto relacionado es el de **camino** que representa la secuencia de nodos intermedios que conectan dos nodos dados. Podemos por ejemplo encontrar el camino más largo a partir de un nodo (en este contexto, la conversación más extensa)

Dado un jsonl conteniendo tweets, podemos construir el grafo utilizando la biblioteca de Python NetworkX

In [ ]:
import sys
import json
from operator import itemgetter
import networkx as nx

#fname = "stream___Rusia2018_WorldCup.jsonl"
#fname = "user_timeline_pottermore.jsonl"
fname = "stream_10000___Rusia2018_WorldCup_saved.jsonl"
with open(fname) as f:
    graph = nx.DiGraph()

    for line in f:
        tweet = json.loads(line)
        if 'id' in tweet:
            graph.add_node(tweet['id'], #este es el único cambo obligatorio, los demás agregan info al nodo
                           tweet=tweet['text'],
                           author=tweet['user']['screen_name'],
                           created_at=tweet['created_at'])

            #Chequeamos si el tweet es una respuesta, para crear un arco
            if tweet['in_reply_to_status_id']:            
                reply_to = tweet['in_reply_to_status_id']
                #Ignoramos si es una auto-respuesta (para no tener en cuenta "monólogos")
                if reply_to in graph and tweet['user']['screen_name'] != graph.node[reply_to]['author']:
                    graph.add_edge(tweet['in_reply_to_status_id'], tweet['id'])
    
    # Imprimimos algunas estadísticas
    print(nx.info(graph))
    
    # Buscamos el tweet con más respuestas
    sorted_replied = sorted(graph.degree(), key=itemgetter(1), reverse=True)
    most_replied_id, replies = sorted_replied[0]
    print("\nEl tweet con más respuestas ({} respuestas):".format(replies))
    print(graph.node[most_replied_id])

    # Buscamos la conversación más extensa
    print("\nConversación más extensa:")
    longest_path = nx.dag_longest_path(graph)
    for tweet_id in longest_path:
        node = graph.node[tweet_id]
        print("{} (por {} el día {})".format(node['tweet'], node['author'],node['created_at']))

## Geolocalizar tweets

Una forma atractiva de visualizar tweets es geolocalizarlos en un mapa, aunque no demasiados tweets tienen esta información.

GeoJSON (http://geojson.org) es un formato común para estructuras de datos geográficos. Permite representar figuras geométricas, características y conjuntos de características. Las figuras geométricas solo contienen información de la forma (por ejemplo Punto, LineString, Polygon). Las características extienden este concepto incorporando propiedades adicionales.

Una estructura GeoJSON es siempre un objeto JSON.

In [ ]:
# Leemos un conjunto de tweets y creamos una estructura GeoJSON
tweets = "stream_10000___Rusia2018_WorldCup_saved.jsonl"
with open(tweets, 'r') as f:
    geo_data = {
        "type": "FeatureCollection",
        "features": []
    }
    for line in f:
        tweet = json.loads(line)
        try:
            if tweet['coordinates']:
                geo_json_feature = {
                    "type": "Feature", 
                    "geometry": {
                        "type": "Point",
                        "coordinates": tweet['coordinates']['coordinates']
                    },
                    "properties": {
                        "text": tweet['text'],
                        "created_at": tweet['created_at']
                    }
                }
                geo_data['features'].append(geo_json_feature)
        except KeyError:
            # Salteamos el tweet en caso de error
            continue
    # Guardamos el archivo GeoJSON
    with open("tweets_map_test.geo.json", 'w') as fout:
        fout.write(json.dumps(geo_data, indent=4))

### Folium

Folium (https://folium.readthedocs.io/en/latest) es una biblioteca Python que permite generar mapas interactivo con muy poco esfuerzo.

Para instalarlo en una distrubución anaconda, ejecutar en la consola `conda install -c conda-forge folium`. En caso de utilizar python de forma independiente ejecutar `pip install folium`

El siguiente código muestra un mapa centrado en Tandil y con dos marcadores, uno para Tandil y otro para Mar del Plata

In [ ]:
import folium 

sample_map = folium.Map(location=[-37.320480, -59.132904], zoom_start=7)
# Marcador para Tandil
tandil_marker = folium.Marker([-37.320480, -59.132904 ], popup='Tandil')
tandil_marker.add_to(sample_map)
# Marcador para Mar del Plata
mardel_marker = folium.Marker([-37.979858, -57.589794], popup='Mar del plata')
mardel_marker.add_to(sample_map)
# Guardamos el mapa en un archivo HTML
sample_map.save("mi_mapa.html")

### Geolocalizamos los tweets

In [ ]:
tweet_map = folium.Map(location=[-37.320480, -59.132904], zoom_start=7)
geojson_layer = folium.GeoJson(open("tweets_map.geo.json",encoding = "utf-8-sig").read())
geojson_layer.add_to(tweet_map)
tweet_map.save("mapa_de_tweets.html")

Si queremos agrupar los tweets cercanos, podemos usar **MarkerCluster**

In [ ]:
from folium.plugins import MarkerCluster

tweet_map = folium.Map(location=[-37.320480, -59.132904], zoom_start=7)
marker_cluster = MarkerCluster().add_to(tweet_map)
geojson_layer = folium.GeoJson(open("tweets_map.geo.json",encoding = "utf-8-sig").read())
geojson_layer.add_to(marker_cluster)
tweet_map.save("mapa_de_tweets_agrupados.html")

Podemos también agregar información adicional a cada marcador para permitir, por ejemplo, ver el tweet exacto asociado a cada marcador.

Un punto a aclarar para esto es que el objeto Marker interpreta las coordenadas como [Latitud, Longitud] mientras que GeoJSON utiliza el formato [Longitud, Latitud]. Es por esto que el arreglo de cooredenadas es invertido (*reverse()*) antes de definir el marcador

In [ ]:
from folium.plugins import MarkerCluster

tweet_map = folium.Map(location=[50, 5], zoom_start=5)
marker_cluster = MarkerCluster().add_to(tweet_map)
geodata = json.load(open("tweets_map.geo.json"))
for tweet in geodata['features']:
    tweet['geometry']['coordinates'].reverse()
    marker = folium.Marker(tweet['geometry']['coordinates'], popup=tweet['properties']['text'])
    marker.add_to(marker_cluster)
tweet_map.save("mapa_de_tweets_agrupados_con_tweets.html")

# Práctico 3
1. Definir el "perfil de publicaciones" de un usuario a partir de las publicaciones realizadas.
2. Definir el perfil de intereses de un usuario a partir de la agregación de los "perfiles de publicación" de la gente que sigue
3. Aplicar un algoritmo de clustering e intentar deducir los temas de interés de un usuario dado a partir de los resultados del mismo
4. Probar para varios usuarios